# Finding Similar Movies

We'll start by loading up the MovieLens dataset. Using Pandas, we can very quickly load the rows of the u.data and u.item files that we care about, and merge them together so we can work with movie names instead of ID's. (In a real production job, you'd stick with ID's and worry about the names at the display layer to make things more efficient. But this lets us understand what's going on better for now.)

In [3]:
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('D:/7.DataScience/DataScience-Python3/ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3), encoding="ISO-8859-1")

m_cols = ['movie_id', 'title']
movies = pd.read_csv('D:/7.DataScience/DataScience-Python3/ml-100k/u.item', sep='|', names=m_cols, usecols=range(2), encoding="ISO-8859-1")

ratings = pd.merge(movies, ratings)


In [4]:
ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


Now the amazing pivot_table function on a DataFrame will construct a user / movie rating matrix. Note how NaN indicates missing data - movies that specific users didn't rate.

In [5]:
movieRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
movieRatings.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's extract a Series of users who rated Star Wars:

In [6]:
starWarsRatings = movieRatings['Star Wars (1977)']
starWarsRatings.head()

user_id
0    5.0
1    5.0
2    5.0
3    NaN
4    5.0
Name: Star Wars (1977), dtype: float64

Pandas' corrwith function makes it really easy to compute the pairwise correlation of Star Wars' vector of user rating with every other movie! After that, we'll drop any results that have no data, and construct a new DataFrame of movies and their correlation score (similarity) to Star Wars:

In [7]:
similarMovies = movieRatings.corrwith(starWarsRatings)
similarMovies = similarMovies.dropna()
df = pd.DataFrame(similarMovies)
df.head(10)

C:\Users\ACER\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\numpy\lib\function_base.py:3159: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\ACER\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\numpy\lib\function_base.py:3093: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


,0
title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398
2 Days in the Valley (1996),0.066654
"20,000 Leagues Under the Sea (1954)",0.289768
2001: A Space Odyssey (1968),0.230884
"39 Steps, The (1935)",0.106453


(That warning is safe to ignore.) Let's sort the results by similarity score, and we should have the movies most similar to Star Wars! Except... we don't. These results make no sense at all! This is why it's important to know your data - clearly we missed something important.

In [8]:
similarMovies.sort_values(ascending=False)

title
Hollow Reed (1996)                                                                   1.000000
Stripes (1981)                                                                       1.000000
Full Speed (1996)                                                                    1.000000
Good Man in Africa, A (1994)                                                         1.000000
Mondo (1996)                                                                         1.000000
Line King: Al Hirschfeld, The (1996)                                                 1.000000
Beans of Egypt, Maine, The (1994)                                                    1.000000
Twisted (1996)                                                                       1.000000
Outlaw, The (1943)                                                                   1.000000
Hurricane Streets (1998)                                                             1.000000
Scarlet Letter, The (1926)                            

Our results are probably getting messed up by movies that have only been viewed by a handful of people who also happened to like Star Wars. So we need to get rid of movies that were only watched by a few people that are producing spurious results. Let's construct a new DataFrame that counts up how many ratings exist for each movie, and also the average rating while we're at it - that could also come in handy later.

In [9]:
import numpy as np
movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})
movieStats.head()

rating          
                            size      mean
title                                     
'Til There Was You (1997)      9  2.333333
1-900 (1994)                   5  2.600000
101 Dalmatians (1996)        109  2.908257
12 Angry Men (1957)          125  4.344000
187 (1997)                    41  3.024390

Let's get rid of any movies rated by fewer than 100 people, and check the top-rated ones that are left:

In [10]:
popularMovies = movieStats['rating']['size'] >= 100
movieStats[popularMovies].sort_values([('rating', 'mean')], ascending=False)[:15]

rating          
                                         size      mean
title                                                  
Close Shave, A (1995)                     112  4.491071
Schindler's List (1993)                   298  4.466443
Wrong Trousers, The (1993)                118  4.466102
Casablanca (1942)                         243  4.456790
Shawshank Redemption, The (1994)          283  4.445230
Rear Window (1954)                        209  4.387560
Usual Suspects, The (1995)                267  4.385768
Star Wars (1977)                          584  4.359589
12 Angry Men (1957)                       125  4.344000
Citizen Kane (1941)                       198  4.292929
To Kill a Mockingbird (1962)              219  4.292237
One Flew Over the Cuckoo's Nest (1975)    264  4.291667
Silence of the Lambs, The (1991)          390  4.289744
North by Northwest (1959)                 179  4.284916
Godfather, The (1972)                     413  4.283293

100 might still be too low, but these results look pretty good as far as "well rated movies that people have heard of." Let's join this data with our original set of similar movies to Star Wars:

In [11]:
df = movieStats[popularMovies].join(pd.DataFrame(similarMovies, columns=['similarity']))

C:\Users\ACER\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\pandas\core\reshape\merge.py:551: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [12]:
df.head()

,"(rating, size)","(rating, mean)",similarity
title,,,
101 Dalmatians (1996),109,2.908257,0.211132
12 Angry Men (1957),125,4.344000,0.184289
2001: A Space Odyssey (1968),259,3.969112,0.230884
Absolute Power (1997),127,3.370079,0.085440
"Abyss, The (1989)",151,3.589404,0.203709


And, sort these new results by similarity score. That's more like it!

In [13]:
df.sort_values(['similarity'], ascending=False)[:15]

,"(rating, size)","(rating, mean)",similarity
title,,,
Star Wars (1977),584,4.359589,1.000000
"Empire Strikes Back, The (1980)",368,4.206522,0.748353
Return of the Jedi (1983),507,4.007890,0.672556
Raiders of the Lost Ark (1981),420,4.252381,0.536117
Austin Powers: International Man of Mystery (1997),130,3.246154,0.377433
"Sting, The (1973)",241,4.058091,0.367538
Indiana Jones and the Last Crusade (1989),331,3.930514,0.350107
Pinocchio (1940),101,3.673267,0.347868
"Frighteners, The (1996)",115,3.234783,0.332729


Ideally we'd also filter out the movie we started from - of course Star Wars is 100% similar to itself. But otherwise these results aren't bad.

## Activity

100 was an arbitrarily chosen cutoff. Try different values - what effect does it have on the end results?

In [14]:
ToyStoryRatings = movieRatings['Toy Story (1995)']
ToyStoryRatings.head()

user_id
0    NaN
1    5.0
2    4.0
3    NaN
4    NaN
Name: Toy Story (1995), dtype: float64

In [15]:
similarities = movieRatings.corrwith(ToyStoryRatings)
similarities = similarities.dropna()
df = pd.DataFrame(similarities)
df.head(10)

C:\Users\ACER\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\numpy\lib\function_base.py:3159: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\ACER\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\numpy\lib\function_base.py:3093: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


,0
title,
'Til There Was You (1997),0.534522
101 Dalmatians (1996),0.232118
12 Angry Men (1957),0.334943
187 (1997),0.651857
2 Days in the Valley (1996),0.162728
"20,000 Leagues Under the Sea (1954)",0.328472
2001: A Space Odyssey (1968),-0.069060
"39 Steps, The (1935)",0.150055
8 1/2 (1963),-0.117259


In [16]:
similarities.sort_values(ascending=False)

title
Scarlet Letter, The (1995)                                                           1.000000
Albino Alligator (1996)                                                              1.000000
Stranger, The (1994)                                                                 1.000000
Substance of Fire, The (1996)                                                        1.000000
Roseanna's Grave (For Roseanna) (1997)                                               1.000000
Newton Boys, The (1998)                                                              1.000000
Reckless (1995)                                                                      1.000000
Guantanamera (1994)                                                                  1.000000
Late Bloomers (1996)                                                                 1.000000
Wooden Man's Bride, The (Wu Kui) (1994)                                              1.000000
Toy Story (1995)                                      

In [20]:
import numpy as np
movieStats = ratings.groupby('title').agg({'rating' : [np.size, np. mean]})
movieStats.head()

rating          
                            size      mean
title                                     
'Til There Was You (1997)      9  2.333333
1-900 (1994)                   5  2.600000
101 Dalmatians (1996)        109  2.908257
12 Angry Men (1957)          125  4.344000
187 (1997)                    41  3.024390

In [32]:
popularMovies = movieStats['rating']['size'] >= 100
movieStats[popularMovies].sort_values([('rating', 'mean')], ascending=False)

rating          
                                                     size      mean
title                                                              
Close Shave, A (1995)                                 112  4.491071
Schindler's List (1993)                               298  4.466443
Wrong Trousers, The (1993)                            118  4.466102
Casablanca (1942)                                     243  4.456790
Shawshank Redemption, The (1994)                      283  4.445230
Rear Window (1954)                                    209  4.387560
Usual Suspects, The (1995)                            267  4.385768
Star Wars (1977)                                      584  4.359589
12 Angry Men (1957)                                   125  4.344000
Citizen Kane (1941)                                   198  4.292929
To Kill a Mockingbird (1962)                          219  4.292237
One Flew Over the Cuckoo's Nest (1975)                264  4.291667
Silence of the Lambs, The (1991)                      390  4.289744
North by Northwest (1959)                             179  4.284916
Godfather, The (1972)                                 413  4.283293
Secrets & Lies (1996)                                 162  4.265432
Good Will Hunting (1997)                              198  4.262626
Manchurian Candidate, The (1962)                      131  4.259542
Dr. Strangelove or: How I Learned to Stop Worry...    194  4.252577
Raiders of the Lost Ark (1981)                        420  4.252381
Vertigo (1958)                                        179  4.251397
Titanic (1997)                                        350  4.245714
Lawrence of Arabia (1962)                             173  4.231214
Maltese Falcon, The (1941)                            138  4.210145
Empire Strikes Back, The (1980)                       368  4.206522
Boot, Das (1981)                                      201  4.203980
Sling Blade (1996)                                    136  4.198529
As Good As It Gets (1997)                             112  4.196429
Godfather: Part II, The (1974)                        209  4.186603
African Queen, The (1951)                             152  4.184211
...                                                   ...       ...
Ace Ventura: Pet Detective (1994)                     103  3.048544
Conan the Barbarian (1981)                            107  3.046729
Grumpier Old Men (1995)                               148  3.040541
Star Trek: The Motion Picture (1979)                  117  3.034188
Broken Arrow (1996)                                   254  3.031496
First Wives Club, The (1996)                          160  3.018750
Net, The (1995)                                       120  3.008333
Evita (1996)                                          259  2.980695
Natural Born Killers (1994)                           128  2.953125
Lost World: Jurassic Park, The (1997)                 158  2.943038
Dante's Peak (1997)                                   240  2.933333
Nutty Professor, The (1996)                           163  2.914110
101 Dalmatians (1996)                                 109  2.908257
Father of the Bride Part II (1995)                    128  2.898438
Mars Attacks! (1996)                                  217  2.847926
Multiplicity (1996)                                   134  2.835821
Alien 3 (1992)                                        100  2.830000
Volcano (1997)                                        219  2.808219
Waterworld (1995)                                     102  2.803922
Beavis and Butt-head Do America (1996)                156  2.788462
Mimic (1997)                                          101  2.742574
Down Periscope (1996)                                 101  2.702970
George of the Jungle (1997)                           162  2.685185
Batman Returns (1992)                                 142  2.683099
Batman Forever (1995)                                 114  2.666667
Spawn (1997)                           

In [37]:
df = movieStats[popularMovies].join(pd.DataFrame(similarities, columns=['similarity']))

C:\Users\ACER\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\pandas\core\reshape\merge.py:551: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [39]:
df.head()

,"(rating, size)","(rating, mean)",similarity
title,,,
101 Dalmatians (1996),109,2.908257,0.232118
12 Angry Men (1957),125,4.344000,0.334943
2001: A Space Odyssey (1968),259,3.969112,-0.069060
Absolute Power (1997),127,3.370079,0.318580
"Abyss, The (1989)",151,3.589404,0.329058


In [42]:
df.sort_values(['similarity'], ascending=False)[:10]

,"(rating, size)","(rating, mean)",similarity
title,,,
Toy Story (1995),452,3.878319,1.000000
"Craft, The (1996)",104,3.115385,0.549100
Down Periscope (1996),101,2.702970,0.457995
Miracle on 34th Street (1994),101,3.722772,0.456291
G.I. Jane (1997),175,3.360000,0.454756
Amistad (1997),124,3.854839,0.449915
Beauty and the Beast (1991),202,3.792079,0.442960
"Mask, The (1994)",129,3.193798,0.432855
Cinderella (1950),129,3.581395,0.428372
